In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comlitrobertalargescript/config.json
/kaggle/input/comlitrobertalargescript/merges.txt
/kaggle/input/comlitrobertalargescript/tokenizer.json
/kaggle/input/comlitrobertalargescript/vocab.json
/kaggle/input/comlitrobertalargescript/tokenizer_config.json
/kaggle/input/comlitrobertalargescript/pytorch_model.bin
/kaggle/input/comlitrobertalargescript/special_tokens_map.json
/kaggle/input/tokenizer/roberta_tk.pt
/kaggle/input/tokenizer/bert_cased_tk.pt
/kaggle/input/commonlitreadabilityprize/sample_submission.csv
/kaggle/input/commonlitreadabilityprize/train.csv
/kaggle/input/commonlitreadabilityprize/test.csv
/kaggle/input/comlitmldata/embed4.bin
/kaggle/input/comlitmldata/mldata.csv
/kaggle/input/comlitmldata/embed3.bin
/kaggle/input/comlitmldata/embed2.bin
/kaggle/input/comlitmldata/embed1.bin
/kaggle/input/cmlit-fold/new_kfold.csv
/kaggle/input/cmlit-fold/train_data.csv
/kaggle/input/comlitrobertabasescript/pre_config.json.backup
/kaggle/input/comlitrobertabasescript/config

In [2]:
import os
import gc
import sys
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from joblib import dump, load

import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import init
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler, Sampler
from transformers import AutoModel, AutoTokenizer, AutoConfig

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

from time import time
from colorama import Fore, Back, Style

r_ = Fore.RED
b_ = Fore.BLUE
g_ = Fore.GREEN
y_ = Fore.YELLOW
w_ = Fore.WHITE
bb_ = Back.BLACK
by_ = Back.YELLOW
sr_ = Style.RESET_ALL

## Global config

In [3]:
class Config:
    epochs = 3
    batch_size = 16
    test_batch = 32
    
    device = 'cuda'
    seed = 42
    max_len = 256
    lr = 2e-5
    weight_decay = 0.01
    
    num_labels = 1

## Dataset

In [4]:
class RoBERTaDataset(Dataset):
    def __init__(self, df, tokenizer, for_test=False):
        super().__init__()
        self.text = df['excerpt'].values
        self.for_test = for_test
        if not for_test:
            self.target = df['target'].values
        self.max_len = Config.max_len
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        text = ' '.join(text.split())
        inputs = self.tokenizer.encode_plus(text,
                                            None,
                                            truncation=True,
                                            add_special_tokens=True,
                                            max_length=self.max_len,
                                            padding='max_length')

        if not self.for_test:
            return {
                'input_ids':
                    torch.tensor(inputs['input_ids'], dtype=torch.long),
                'attention_mask':
                    torch.tensor(inputs['attention_mask'], dtype=torch.long),
                'label':
                    torch.tensor(self.target[index], dtype=torch.float)
            }
        else:
            return {
                'input_ids':
                    torch.tensor(inputs['input_ids'], dtype=torch.long),
                'attention_mask':
                    torch.tensor(inputs['attention_mask'], dtype=torch.long)
            }

### Model Attn 1

In [5]:
# CLRPModel: 467    BaseOne: 474  Base 2: 475     
class AttnOneConifg:
    model_name = 'roberta-base'
    pretrained_model_path = '/kaggle/input/comlitrobertabasescript/'
    
    epochs = 3
    batch_size = 16
    test_batch = 32
    
    seed = 42
    max_len = 256
    lr = 2e-5
    weight_decay = 0.01

In [6]:
import torch
import torch.nn as nn

# Using Model
# Base Model One         Base Model Two
class AttentionHead_Ori(nn.Module):
    def __init__(self, h_size, hidden_dim=512):
        super().__init__()
        self.W = nn.Linear(h_size, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

class CLRPModel(nn.Module):
    def __init__(self):
        super(CLRPModel, self).__init__()
        config = AutoConfig.from_pretrained(AttnOneConifg.pretrained_model_path)
        config.update({"output_hidden_states":True,
                        "hidden_dropout_prob": 0.0,
                        "layer_norm_eps": 1e-7})
        self.h_size = config.hidden_size
        self.transformer = AutoModel.from_pretrained(AttnOneConifg.pretrained_model_path, config=config)
        self.head = AttentionHead_Ori(self.h_size)
        self.linear = nn.Linear(self.h_size, 1)

    def forward(self, input_ids, attention_mask):
        transformer_out = self.transformer(input_ids, attention_mask)
        context = self.head(transformer_out.last_hidden_state)
        x = self.linear(context)
        return x, context

In [7]:
# 467
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()
        config = AutoConfig.from_pretrained(AttnOneConifg.pretrained_model_path)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(AttnOneConifg.pretrained_model_path, config=config)  
        self.attention = nn.Sequential(            
            nn.Linear(768, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        
        self.regressor = nn.Sequential(nn.Linear(768, 1))
        
    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids, attention_mask)        
        last_hidden_states = roberta_output.hidden_states[-1]
        
        weights = self.attention(last_hidden_states)
        context_vector = torch.sum(weights * last_hidden_states, dim=1)        
        
        return self.regressor(context_vector), context_vector

### Model MeanPooling Large and MeanEmbedding

MeanEmbedding完全可以和Model MeanPooling Large合在一起

- MeanEmbedding V1 + SVM 473

In [8]:
# Mean Model
class MeanLargeConfig:
    model_name = 'roberta-large'
    pretrained_model_path = '../input/comlitrobertalargescript'
    
    epochs = 3
    batch_size = 16
    test_batch = 32
    
    seed = 42
    max_len = 256
    lr = 2e-5
    weight_decay = 0.01
    
    head_hidden = 512
    
    use_multi_sample_dropout = True # this model didn`t help

In [9]:
# V1 large :  inference model  同时输出 logits 和 pool_out embedding，可以选择 如何取舍两个输出。
#             MeanEmbedding V1 + SVM 473
#                    logits          472
class MeanPooling(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        
    def forward(self, hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_state.size()).float()
        sum_embeddings = torch.sum(hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

    
class MeanModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.config = AutoConfig.from_pretrained(MeanLargeConfig.pretrained_model_path)
        self.config.update({"output_hidden_states": True,
                            "hidden_dropout_prob": 0.0,
                            "attention_probs_dropout_prob": 0.1,
                            "layer_norm_eps": 1e-7}) 
        self.roberta = AutoModel.from_pretrained(MeanLargeConfig.pretrained_model_path,
                                                 config=self.config)
        self.layer_norm = nn.LayerNorm(self.config.hidden_size, eps=1e-7)
        self.pooler = MeanPooling(self.config.hidden_size)

        self.low_dropout = nn.Dropout(0.1)
        self.dropout = nn.Dropout(p=0.5)
        self.regressor = nn.Linear(self.config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        
        ## MeanPooling
        hidden_states = outputs[0]
        pool_out = self.pooler(hidden_states, attention_mask)
        # pool_out = self.low_dropout(pool_out)
        
        # didn`t help
        logits = torch.mean(torch.stack([self.regressor(self.dropout(pool_out)) for _ in range(5)], dim=0), dim=0)

        return (logits, pool_out)

### Attn v2

In [10]:
class AttConfig1:
    model_name = 'roberta-large'
    pretrained_model_path = '../input/comlitrobertalargescript'
    
    output_hidden_states = True
    epochs = 3
    num_labels = 1
    
    device = 'cuda'
    
    seed = 42
    max_len = 256
    lr = 2e-5
    weight_decay = 0.01
    head_hidden = 512
    
    warmup_steps = 50

In [11]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, hidden_state, attention_mask):
        att = torch.tanh(self.W(hidden_state))
        score = self.V(att)

        mask_expanded = attention_mask.unsqueeze(-1)
        score[~mask_expanded] = -1e9

        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * hidden_state
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector
    
    
class AttModel(nn.Module):
    def __init__(self, config, attn_type='tradition'):
        super().__init__()
        self.config = AutoConfig.from_pretrained(config.pretrained_model_path)
        self.config.update({
                "output_hidden_states": True,
                "hidden_dropout_prob": 0.0,
                "attention_probs_dropout_prob": 0.1,
                "layer_norm_eps": 1e-7
                }) 
        self.roberta = AutoModel.from_pretrained(config.pretrained_model_path,
                                                 config=self.config)
        self.head = AttentionHead(self.config.hidden_size, config.head_hidden)
        
        self.layer_norm = nn.LayerNorm(self.config.hidden_size, eps=1e-5)
        self.regressor = nn.Linear(self.config.hidden_size, config.num_labels)
        
        self.dropout = nn.Dropout(p=0.1)
        self.m_dropout = nn.Dropout(p=0.5)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        hidden_states = outputs[2][-1]

        x = self.head(hidden_states, attention_mask)
        logits = self.regressor(x)

        return logits, x

### Attn 3

In [12]:
class AttConfig2:
    model_name = 'roberta-base'
    pretrained_model_path = '../input/comlitrobertabasescript'
    
    output_hidden_states = True
    epochs = 3
    num_labels = 1
    
    device = 'cuda'
    
    seed = 42
    max_len = 256
    lr = 2e-5
    weight_decay = 0.01
    head_hidden = 512
    
    warmup_steps = 50

In [13]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, hidden_state, attention_mask):

        att = torch.tanh(self.W(hidden_state))
        score = self.V(att)

        mask_expanded = attention_mask.unsqueeze(-1)
        score[~mask_expanded] = -1e9

        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * hidden_state
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector
    
    
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x *( torch.tanh(F.softplus(x)))
    
    
class AttModel2(nn.Module):
    def __init__(self, config, attn_type='tradition'):
        super().__init__()
        self.config = AutoConfig.from_pretrained(config.pretrained_model_path)
        self.config.update({
                "output_hidden_states": True,
                "hidden_dropout_prob": 0.0,
                "attention_probs_dropout_prob": 0.1,
                "layer_norm_eps": 1e-7
                }) 
        self.roberta = AutoModel.from_pretrained(config.pretrained_model_path,
                                                 config=self.config)
        self.head = AttentionHead(self.config.hidden_size, config.head_hidden)

        self.layer_norm = nn.LayerNorm(self.config.hidden_size, eps=1e-5)
        self.regressor = nn.Sequential(
                nn.Linear(self.config.hidden_size, 512),
                Mish(),
                nn.Linear(512, 1)
        )
        
        self.dropout = nn.Dropout(p=0.1)
        self.m_dropout = nn.Dropout(p=0.5)
        

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        hidden_states = outputs[2][-1]

        x = self.head(hidden_states, attention_mask)

        x = self.layer_norm(x)
        logits = self.regressor(x)
        
        return logits, x

### Mean v2

In [14]:
# Mean Model
class MeanV2BaseConfig:
    model_name = 'roberta-base'
    pretrained_model_path = '../input/comlitrobertabasescript'
    
    epochs = 3
    batch_size = 16
    test_batch = 32
    
    seed = 42
    max_len = 256
    lr = 2e-5
    weight_decay = 0.01
    
    head_hidden = 512

In [15]:
# Mean Large Model
class MeanV2LargeConfig:
    model_name = 'roberta-large'
    pretrained_model_path = '../input/comlitrobertalargescript'
    
    epochs = 3
    batch_size = 16
    test_batch = 32
    
    seed = 42
    max_len = 256
    lr = 2e-5
    weight_decay = 0.01
    
    head_hidden = 512

In [16]:
# V2 base :  inference model

class MeanPooling(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        
    def forward(self, hidden_state, attention_mask):
        # last_hidden_state = outputs[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_state.size()).float()
        sum_embeddings = torch.sum(hidden_state * input_mask_expanded, 1)

        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        
        return mean_embeddings


#Mish - "Mish: A Self Regularized Non-Monotonic Neural Activation Function"
#https://arxiv.org/abs/1908.08681v1
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x *( torch.tanh(F.softplus(x)))

    
class MeanModel_v2(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = AutoConfig.from_pretrained(config.pretrained_model_path)
        self.config.update({
                "output_hidden_states": True,
                "hidden_dropout_prob": 0.0,
                "attention_probs_dropout_prob": 0.1,
                "layer_norm_eps": 1e-7
                }) 
        self.roberta = AutoModel.from_pretrained(config.pretrained_model_path,
                                                 config=self.config)
        
        self.pooler = MeanPooling(self.config.hidden_size)
        self.layer_norm = nn.LayerNorm(self.config.hidden_size, eps=1e-5)
        
        self.dropout = nn.Dropout(0.5)
        self.regressor = nn.Sequential(
                nn.Linear(self.config.hidden_size, 512),
                Mish(),
                nn.Linear(512, 1)
        )
        
        self.std = 0.02
        self._init_weights(self.regressor)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            # module.weight.data.normal_(mean=0.0, std=self.std)
            init.kaiming_normal_(module.weight, mode='fan_in')
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        
        ## MeanPooling
        hidden_states = outputs[0]
        pool_out = self.pooler(hidden_states, attention_mask)
        # pool_out = self.dropout(pool_out)
        pool_out = self.layer_norm(pool_out)
        logits = self.regressor(pool_out)

        return logits, pool_out

### Infer func

In [17]:
def get_test_data(df):
    tokenizer = torch.load('/kaggle/input/tokenizer/roberta_tk.pt') 
    test_dataset = RoBERTaDataset(df, tokenizer, for_test=True)
    test_loader = DataLoader(test_dataset, batch_size=32,
                             num_workers=4, shuffle=False, pin_memory=True,
                             drop_last=False)
    return test_loader

In [18]:
def reset_memory():
    gc.collect()
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

In [19]:
def inference(test_dataloader, model_dirs, model=None, n_models=5, ckpt_bias=0, with_embedding=False):
    models_preds = []
    models_embedding = []
    for model_num in range(n_models):
        print(f'{by_}{r_}  >>> Inference # {model_num+1}/{n_models}  {sr_}')
        torch.cuda.synchronize()

        # load
        model_path = model_dirs[model_num]
        print(f" ### Using {model_path}")
        if model:
            model.load_state_dict(torch.load(model_path, map_location=Config.device))
        else:
            model = torch.load(model_path)
        model.to(Config.device)

        # predict
        fold_preds = []
        embeddings = []
        model.eval()
        with torch.no_grad():
            for step, batch in enumerate(test_dataloader):
                sent_id, mask = batch['input_ids'].to(Config.device), batch['attention_mask'].to(Config.device)
                preds = model(sent_id, mask)
                
                if with_embedding and len(preds) == 2:
                    preds, embed = preds[0], preds[1]
                    embed = embed.detach().cpu().numpy()
                    embeddings.extend(embed)
                if len(preds) == 2:
                    preds = preds[0]
                fold_preds += preds.flatten().cpu().tolist()

        # records
        models_preds.append(fold_preds)
        if with_embedding:
            models_embedding.append(np.array(embeddings))

        if not model:  # load_state_dict 方式，不能在这里删除
            del model
            gc.collect()
            torch.cuda.synchronize()
            torch.cuda.empty_cache()

        print(f'! Model Complete. ++++++++++')
    print()

    # output
    models_preds = np.array(models_preds).mean(axis=0)
    if not with_embedding:
        return models_preds
    else:
        return models_preds, models_embedding

In [20]:
def embedding_svr_test(embeddings, num_pred, bert_nums=5, svr_nfolds=10):
    # SVM predict: 5 SVR model
    results = np.zeros(num_pred)
    for index, X_test in enumerate(embeddings):
        print(f'{by_}{r_}  SVR#{index+1} predicting {sr_}')
        for i in range(svr_nfolds):
            svr = load(save_dir + f'svr_{index}_{i}.bin')
            preds = svr.predict(X_test)
            results += preds
    print(f'SVR Complete.')

    return results / bert_nums / svr_nfolds

In [21]:
# !pip install textstat --no-index --find-links=file:///kaggle/input/textstat-local/textstat 

# import textstat

# def add_textstat_features(df):
#     ### You can add/remove any feature below and it will be used in training and test
#     df['coleman_liau_index'] = df['excerpt'].apply(lambda x: textstat.coleman_liau_index(x))
#     df['flesch_reading_ease'] = df['excerpt'].apply(lambda x: textstat.flesch_reading_ease(x))
#     df['smog_index'] = df['excerpt'].apply(lambda x: textstat.smog_index(x))
#     df['dale_chall_readability_score'] = df['excerpt'].apply(lambda x: textstat.dale_chall_readability_score(x))
#     return df

# def difficult_words_ratio(df, input_col='excerpt', output_col='difficult_words_ratio'):
#     print(f"Applying {output_col} to data set.")
#     df[output_col] = df[input_col].apply(lambda x: textstat.difficult_words(x))
#     df[output_col] = df.apply(lambda x: x[output_col] / textstat.lexicon_count(x[input_col]), axis=1)
#     return df 

# def syllable_ratio(df, input_col='excerpt', output_col='syllable_ratio'):
#     print(f"Applying {output_col} to data set.")
#     df[output_col] = df[input_col].apply(lambda x: textstat.syllable_count(x))
#     df[output_col] = df.apply(lambda x: x[output_col] / textstat.lexicon_count(x[input_col]), axis=1)
#     return df

In [22]:
def save_embeddings(embeds, path):
    import pickle
    with open(path, "wb") as f:
        pickle.dump(embeds, f)

def load_embeddings(path):
    import pickle
    with open(path, "rb") as f:
        emb = pickle.load(f)
    return emb

## MAIN

In [23]:
# test_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
test_df = pd.read_csv("/kaggle/input/cmlit-fold/train_data.csv")
test_df['excerpt'] = test_df['excerpt'].apply(lambda x: x.replace('\n',' '))



test_dataloader = get_test_data(test_df)
#### LitModel: 467
Lit = [f'../input/comlitothers/model_{i + 1}.bin' for i in range(5)]
litmodel = LitModel()  # if use load_state_dict, init model
pred_litm, embed1 = inference(test_dataloader, Lit, model=litmodel, ckpt_bias=1,with_embedding=True)
test_df["m1"] = pred_litm
save_embeddings(embed1, "./embed1.bin")

del litmodel, test_dataloader
reset_memory()


# #### mean large v2: 0.469  Nice
# MeanL_3 = [ f'/kaggle/input/newmeanlarge/model_{i}.bin' for i in range(5)]
# config = MeanV2LargeConfig()
# meanmodel3 = MeanModel_v2(config)
# pred_mean_v3, embed2 = inference(test_dataloader, MeanL_3, model=meanmodel3, with_embedding=True)
# # svr_preds = embedding_svr_test(embeddings, len(test_df))
# test_df["m2"] = pred_mean_v3
# save_embeddings(embed2, "./embed2.bin")

# del meanmodel3
# reset_memory()


test_dataloader = get_test_data(test_df)
#### attn large 1:  0.464
AttL_1 = [ f'/kaggle/input/largeattnlit/model_{i}.bin' for i in range(5)]
config = AttConfig1()
attmodel1 = AttModel(config)
pred_attL_v1, embed3 = inference(test_dataloader, AttL_1, model=attmodel1, with_embedding=True)
# svr_preds = embedding_svr_test(embeddings, len(test_df))
test_df["m3"] = pred_attL_v1
save_embeddings(embed3, "./embed3.bin")

del attmodel1, test_dataloader
reset_memory()



test_dataloader = get_test_data(test_df)
#### mean large v2 reinit: 0.466
MeanL_4 = [ f'/kaggle/input/meanlargereinit/model_{i}.bin' for i in range(5)]
config = MeanV2LargeConfig()
meanmodel4 = MeanModel_v2(config)
pred_mean_v4, embed4 = inference(test_dataloader, MeanL_4, model=meanmodel4, with_embedding=True)
# svr_preds = embedding_svr_test(embeddings, len(test_df))
test_df["m4"] = pred_mean_v4
save_embeddings(embed4, "./embed4.bin")

del meanmodel4, test_dataloader
reset_memory()



test_dataloader = get_test_data(test_df)
#### attn large reinit 2:  0.467
AttL_2 = [ f'/kaggle/input/attlargereinit/model_{i}.bin' for i in range(5)]
config = AttConfig1()
attmodel2 = AttModel(config)
pred_attL_v2, embed2  = inference(test_dataloader, AttL_2, model=attmodel2, with_embedding=True)
# svr_preds = embedding_svr_test(embeddings, len(test_df))
test_df["m2"] = pred_attL_v2
save_embeddings(embed2, "./embed2.bin")

del attmodel2, test_dataloader
reset_memory()

Some weights of the model checkpoint at /kaggle/input/comlitrobertabasescript/ were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /kaggle/input/comlitrobertabasescript/ and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predict

  >>> Inference # 1/5  
 ### Using ../input/comlitothers/model_1.bin
! Model Complete. ++++++++++
  >>> Inference # 2/5  
 ### Using ../input/comlitothers/model_2.bin
! Model Complete. ++++++++++
  >>> Inference # 3/5  
 ### Using ../input/comlitothers/model_3.bin
! Model Complete. ++++++++++
  >>> Inference # 4/5  
 ### Using ../input/comlitothers/model_4.bin
! Model Complete. ++++++++++
  >>> Inference # 5/5  
 ### Using ../input/comlitothers/model_5.bin
! Model Complete. ++++++++++



Some weights of the model checkpoint at ../input/comlitrobertalargescript were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/comlitrobertalargescript and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

  >>> Inference # 1/5  
 ### Using /kaggle/input/largeattnlit/model_0.bin
! Model Complete. ++++++++++
  >>> Inference # 2/5  
 ### Using /kaggle/input/largeattnlit/model_1.bin
! Model Complete. ++++++++++
  >>> Inference # 3/5  
 ### Using /kaggle/input/largeattnlit/model_2.bin
! Model Complete. ++++++++++
  >>> Inference # 4/5  
 ### Using /kaggle/input/largeattnlit/model_3.bin
! Model Complete. ++++++++++
  >>> Inference # 5/5  
 ### Using /kaggle/input/largeattnlit/model_4.bin
! Model Complete. ++++++++++



Some weights of the model checkpoint at ../input/comlitrobertalargescript were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/comlitrobertalargescript and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

  >>> Inference # 1/5  
 ### Using /kaggle/input/meanlargereinit/model_0.bin
! Model Complete. ++++++++++
  >>> Inference # 2/5  
 ### Using /kaggle/input/meanlargereinit/model_1.bin
! Model Complete. ++++++++++
  >>> Inference # 3/5  
 ### Using /kaggle/input/meanlargereinit/model_2.bin
! Model Complete. ++++++++++
  >>> Inference # 4/5  
 ### Using /kaggle/input/meanlargereinit/model_3.bin
! Model Complete. ++++++++++
  >>> Inference # 5/5  
 ### Using /kaggle/input/meanlargereinit/model_4.bin
! Model Complete. ++++++++++



Some weights of the model checkpoint at ../input/comlitrobertalargescript were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../input/comlitrobertalargescript and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

  >>> Inference # 1/5  
 ### Using /kaggle/input/attlargereinit/model_0.bin
! Model Complete. ++++++++++
  >>> Inference # 2/5  
 ### Using /kaggle/input/attlargereinit/model_1.bin
! Model Complete. ++++++++++
  >>> Inference # 3/5  
 ### Using /kaggle/input/attlargereinit/model_2.bin
! Model Complete. ++++++++++
  >>> Inference # 4/5  
 ### Using /kaggle/input/attlargereinit/model_3.bin
! Model Complete. ++++++++++
  >>> Inference # 5/5  
 ### Using /kaggle/input/attlargereinit/model_4.bin
! Model Complete. ++++++++++



In [24]:
# embedding
# features = embed3

# 预测值
# text stats features
# test_df = add_textstat_features(test_df)
# test_df = difficult_words_ratio(test_df)
# test_df = syllable_ratio(test_df)

# print(test_df.head())
# test_df.to_csv("./mldata.csv", index=False)

In [25]:
# import os

# # !pip install kaggle

# os.environ["KAGGLE_USERNAME"] = "racleray"
# os.environ["KAGGLE_KEY"] = "d1c80c062506c912e369893b207eaca9"

# !kaggle datasets metadata racleray/comlitmldata
# # !mv dataset-metadata.json model/

# # 最好不要有文件夹
# !kaggle datasets version -p ./ -m "Updated data base fine"

### ML

In [28]:
embed1 = load_embeddings("./embed1.bin")
print(len(embed1))

embed2 = load_embeddings("./embed2.bin")
print(len(embed2))

embed3 = load_embeddings("./embed3.bin")
print(len(embed3))

embed4 = load_embeddings("./embed4.bin")
print(len(embed4))

5
5
5
5


SVR

In [30]:
# rmse_score SVR
from joblib import dump, load
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold


train_data = test_df

## Reset bins
num_bins = int(np.floor(1 + np.log2(len(train_data))))
train_data.loc[:, 'bins'] = pd.cut(train_data['target'], bins=num_bins, labels=False)

target = train_data['target'].to_numpy()
bins = train_data.bins.to_numpy()

In [31]:
train_data.head()

,id,url_legal,license,excerpt,target,standard_error,kfold,bins,m1,m3,m4,m2
0,f305c0e4a,https://www.africanstorybook.org/,CC BY 4.0,Zonke runs home. It's quite hot. Fortunately t...,0.059705,0.470262,0,8,-0.257335,-0.118455,0.022293,-0.120040
1,b5b6a8c91,NaN,NaN,Then the causes of the war are summed up and t...,-2.467364,0.511015,0,2,-2.249151,-2.206307,-2.206216,-2.287320
2,9a9b134c8,NaN,NaN,One day mousie managed to get his door open an...,0.323143,0.516715,0,8,0.210188,0.306258,0.274027,0.235040
3,f6172c6a5,NaN,NaN,"In a system of arc lighting, however, we have ...",-2.882630,0.532794,0,1,-2.568251,-2.748233,-2.770499,-2.757580
4,84d86d648,NaN,NaN,The nature of incrustation and the evils resul...,-2.519765,0.538096,0,2,-2.828407,-2.721420,-2.867517,-2.889204


In [33]:
def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))


# input whole train_data not shuffled
def embedding_svr_train(df, bins, models_embedding, save_dir="./", emb=1, bert_nums=5, svr_nfolds=10, C=8, kernel='rbf'):
    mean_scores = []
    records = []

    # get embeddings
    #print(models_embedding[0].shape)
    #print(type(models_embedding[0]))    
    print("Embedding got.")    

    
    # SVM training: 5 SVR model
    for index, X in enumerate(models_embedding):
        print(f'{by_}{r_}  SVR#{index+1} training {sr_}')
        scores = []
        model = SVR(C=C, kernel=kernel, gamma='auto')
        # new kfold
        kfold = StratifiedKFold(n_splits=svr_nfolds, shuffle=True, random_state=42)
        for i, (train_idx, valid_idx) in tqdm(enumerate(kfold.split(X, bins))):
#             model = SVR(C=C, kernel=kernel, gamma='auto')
            X_train, y_train = X[train_idx], target[train_idx]
            X_valid, y_valid = X[valid_idx], target[valid_idx]
            
            model.fit(X_train, y_train)
            
            prediction = model.predict(X_valid)
            score = rmse_score(prediction, y_valid)
            scores.append(score)
            print(f'\t\t{y_}SVR {index} Fold {i} , rmse score: {score:.4f} {sr_}')

        os.makedirs(save_dir, exist_ok=True)
        dump(model, save_dir + f'svr_{emb}_{index}.bin')

        mean_score = np.mean(scores)
        print(f'\t{r_}SVR {index} mean rmse score: {mean_score:.4f} {sr_}')
        mean_scores.append(mean_score)
        records.append(scores)

    print(f'Avg rmse score of 5 SVR: {np.mean(mean_scores):.4f}')

    return records

In [34]:
embedding_svr_train(train_data, bins, embed1, emb=1, svr_nfolds=5, C=100) 

0it [00:00, ?it/s]

Embedding got.
  SVR#1 training 


1it [00:05,  5.71s/it]

		SVR 0 Fold 0 , rmse score: 0.3454 


2it [00:11,  5.82s/it]

...

4it [00:23,  5.96s/it]

		SVR 4 Fold 3 , rmse score: 0.3452 


5it [00:29,  5.95s/it]

		SVR 4 Fold 4 , rmse score: 0.3408 
	SVR 4 mean rmse score: 0.3539 
Avg rmse score of 5 SVR: 0.3429


[[0.34542270076541276,
  0.3066241762632316,
  0.3376639678954363,
  0.34089036541837203,
  0.3206861963784917],
 [0.34288948821891163,
  0.3255652466470202,
  0.3442932237216695,
  0.36999307523492914,
  0.3407440016154367],
 [0.35957686789556764,
  0.3316827325149671,
  0.34366729198538715,
  0.3450497578191213,
  0.33155643655330974],
 [0.35447239387690144,
  0.32958864038603186,
  0.34326825974642905,
  0.34992483278055764,
  0.338717355762261],
 [0.37158189102335165,
  0.3512817192865013,
  0.36063067444118924,
  0.34515415649942255,
  0.3407595911639054]]

In [35]:
embedding_svr_train(train_data, bins, embed1, svr_nfolds=5, C=10)

In [36]:
embedding_svr_train(train_data, bins, embed1, emb=1, svr_nfolds=5, C=200)

In [37]:
embedding_svr_train(train_data, bins, embed2, emb=2, svr_nfolds=5, C=200)

0it [00:00, ?it/s]

Embedding got.
  SVR#1 training 


1it [00:10, 10.14s/it]

		SVR 0 Fold 0 , rmse score: 0.3370 


2it [00:20, 10.53s/it]

...

4it [00:44, 10.99s/it]

		SVR 4 Fold 3 , rmse score: 0.3726 


5it [00:54, 10.99s/it]

		SVR 4 Fold 4 , rmse score: 0.3485 
	SVR 4 mean rmse score: 0.3625 
Avg rmse score of 5 SVR: 0.3513


[[0.3369784314466992,
  0.3422228064093938,
  0.34491332569811,
  0.3408109833069153,
  0.3733180779351824],
 [0.3637201404577287,
  0.3610829405629857,
  0.36078279780468137,
  0.3625815295684579,
  0.3479460353247875],
 [0.360134852736254,
  0.3298976342619192,
  0.3570128172001221,
  0.34751204017336307,
  0.3462330229028444],
 [0.3519340318365144,
  0.3192790751929824,
  0.3436934037302124,
  0.35195490690301723,
  0.32814413636124806],
 [0.3640268274635047,
  0.36305882931730327,
  0.3643575559389901,
  0.3725785223150587,
  0.34850652763179374]]

In [38]:
embedding_svr_train(train_data, bins, embed3, emb=3, svr_nfolds=5, C=200)

0it [00:00, ?it/s]

Embedding got.
  SVR#1 training 


1it [00:07,  7.97s/it]

		SVR 0 Fold 0 , rmse score: 0.3009 


2it [00:15,  8.00s/it]

...

4it [00:41, 10.33s/it]

		SVR 4 Fold 3 , rmse score: 0.3658 


5it [00:52, 10.44s/it]

		SVR 4 Fold 4 , rmse score: 0.3508 
	SVR 4 mean rmse score: 0.3640 
Avg rmse score of 5 SVR: 0.3419


[[0.3008846318301386,
  0.2802867726295797,
  0.3061025955906373,
  0.30476754725983973,
  0.3331477800721458],
 [0.3782995575127707,
  0.38201804361946756,
  0.3680116488543579,
  0.36097150579246556,
  0.35403955634611833],
 [0.3282692770367046,
  0.3217849897624967,
  0.3303575141942111,
  0.32949701042460144,
  0.31406496655708693],
 [0.35947327272156576,
  0.33839697890060977,
  0.344034104423757,
  0.35387178075395104,
  0.34001346128670284],
 [0.3653576328940215,
  0.3627181580014248,
  0.37521238826325043,
  0.3658153610399832,
  0.3507540355002089]]

In [39]:
embedding_svr_train(train_data, bins, embed3, svr_nfolds=5, C=250)

In [40]:
embedding_svr_train(train_data, bins, embed4, emb=4, svr_nfolds=5, C=200)

0it [00:00, ?it/s]

Embedding got.
  SVR#1 training 


1it [00:11, 11.33s/it]

		SVR 0 Fold 0 , rmse score: 0.3479 


2it [00:22, 11.49s/it]

...

4it [00:45, 11.38s/it]

		SVR 4 Fold 3 , rmse score: 0.3550 


5it [00:56, 11.29s/it]

		SVR 4 Fold 4 , rmse score: 0.3401 
	SVR 4 mean rmse score: 0.3550 
Avg rmse score of 5 SVR: 0.3467


[[0.3479005349136949,
  0.32679807654805326,
  0.32634154275449173,
  0.35603007301114686,
  0.35592534787324104],
 [0.3528719546925852,
  0.335419015287315,
  0.3387324971504691,
  0.3337402915881854,
  0.33779542310578065],
 [0.36061875525910986,
  0.3417117325496616,
  0.34817651439157987,
  0.34102294879441114,
  0.3432128630570253],
 [0.3639741353603139,
  0.33750324078849714,
  0.34663032419664425,
  0.3562736064353681,
  0.34125973545973404],
 [0.37581420533554266,
  0.34595781378819274,
  0.3578758069246591,
  0.35504601808971337,
  0.34008128574996926]]

In [1]:
# import os

# # !pip install kaggle

# os.environ["KAGGLE_USERNAME"] = ""
# os.environ["KAGGLE_KEY"] = ""

# !kaggle datasets metadata racleray/svrmodel
# # !mv dataset-metadata.json model/

# # 最好不要有文件夹
# !kaggle datasets version -p ./ -m "Updated data base fine"

### Light GBM

过拟合

In [ ]:
# ml_df = pd.read_csv("../input/comlitmldata/mldata.csv")
# ml_df.head()

In [ ]:
# Smaller maxbin: reduces train accuracy but has potential to increase generalization
# Bigger min_data_in_leaf: has potential to reduce overfitting

# params = {
# 'boosting_type': 'gbdt',
# 'objective': 'regression',
# 'metric': 'rmse',
#     # baseline at 100 for min_data_in_leaf
# 'min_data_in_leaf': 100,
#     # baseline at .8
# 'feature_fraction': .8,
#     # baseline at .8
# 'bagging_fraction': 0.8,
# 'bagging_freq': 10,
# 'max_depth': 10,
# 'num_leaves': 32,
# 'learning_rate': 0.05,
#     # baseline at max_bin 256
# "max_bin": 100,
# "n_estimators": 10000,
# }


# params = {
#     'boosting_type': 'gbdt',
#     'metric': 'rmse',
#     'objective': 'regression',
#     'verbose': -1,
#     'learning_rate': 0.05,
#     'max_depth': 10,
#     'feature_pre_filter': False,
#     'lambda_l1': 2.215942517163985,
#     'lambda_l2': 0.0015606472088872934,
#     'num_leaves': 2,
#     'feature_fraction': 0.8999999999999999,
#     'bagging_fraction': 1.0,
#     'bagging_freq': 0,
#     'min_child_samples': 30,
# }


# lgm_data = ml_df.copy()

# cols2remove = ['url_legal'
#                , 'excerpt'
#                , 'id'
#                , 'license'
#                , 'kfold'
#                , 'bins'
#                , 'standard_error'
# #                , 'm2', 'm3', 'm4', 'm1'
#               ]

# lgm_data = lgm_data.drop(columns=cols2remove)


# lgm_data.head()

In [ ]:
# import lightgbm as lgm

# ## Reset bins
# num_bins = int(np.floor(1 + np.log2(len(lgm_data))))
# lgm_data.loc[:, 'bins'] = pd.cut(lgm_data['target'], bins=num_bins, labels=False)

# X = lgm_data.loc[:, lgm_data.columns != 'target'].to_numpy()
# target = lgm_data['target'].to_numpy()
# bins = lgm_data.bins.to_numpy()


# pred = np.zeros(ml_df.shape[0])
# rmses = []
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# for i, (train_idx, valid_idx) in tqdm(enumerate(kfold.split(X, bins))):
#     X_train, y_train = X[train_idx], target[train_idx]
#     X_valid, y_valid = X[valid_idx], target[valid_idx]
    
#     lgm_train_set = lgm.Dataset(data=X_train, label=y_train)
#     lgm_valid_set = lgm.Dataset(data=X_valid, label=y_valid, reference=lgm_train_set)
    
#     model = lgb.train(
#         params,
#         lgm_train_set, 
#         num_boost_round=1000,
#         early_stopping_rounds=10,
#         valid_sets=[lgm_train_set, lgm_valid_set], 
#         verbose_eval=-1
#     )

#     y_pred = model.predict(X_valid)
#     rmse = rmse_score(y_pred, y_valid)
#     rmses.append(rmse)
    
#     # tmp_pred = model.predict(X_test)
#     # pred += tmp_pred / 5
    
# print("\n", "Mean Fold RMSE:", np.mean(rmses))

In [ ]:
if __name__ == "__main__":
    test_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
    test_df['excerpt'] = test_df['excerpt'].apply(lambda x: x.replace('\n',' '))
    test_dataloader = get_test_data(test_df)


    ################### Predict ########################
    ################## load model
    #### Base CLRPModel 
#     CLRP_1 = [f'../input/comlitmodelone/best_model_{i}.pt' for i in range(5)]
#     preds_c1 = inference(test_dataloader, CLRP_1, with_embedding=False)
#     reset_memory()


    #### Base CLRPModel 
#     CLRP_2 = [f'../input/comlitbase2/best_model_{i}.pt' for i in range(5)]
#     preds_c2 = inference(test_dataloader, CLRP_2, with_embedding=False)
#     reset_memory()



    ################## load_state_dict
    #### LitModel: 
#     Lit = [f'../input/comlitothers/model_{i + 1}.bin' for i in range(5)]
#     litmodel = LitModel()  # if use load_state_dict, init model
#     pred_lit = inference(test_dataloader, Lit, model=litmodel, ckpt_bias=1)
#     del litmodel
#     reset_memory()


    #### mean large v1: 472  ;    MeanEmbedding V1 + SVM 473
#     MeanL_1 = [ f'../input/clrobertalarger/model_{i}.bin' for i in range(5)]
#     meanmodel = MeanModel()
#     pred_mean_v1, embeddings = inference(test_dataloader, MeanL_1, model=meanmodel, with_embedding=True)
#     svr_preds = embedding_svr_test(embeddings, len(test_df))
#     del meanmodel
#     reset_memory()


    #### mean base v2: 
#     MeanB_2 = [ f'../input/newmeanbase/model_{i}.bin' for i in range(5)]
#     config = MeanV2BaseConfig()
#     meanmodel2 = MeanModel_v2(config)
#     pred_mean_v2 = inference(test_dataloader, MeanB_2, model=meanmodel2, with_embedding=False)
#     # svr_preds = embedding_svr_test(embeddings, len(test_df))
#     del meanmodel2
#     reset_memory()


    #### mean base fgm v2: 
#     MeanBF_2 = [ f'../input/newmeanbasefgm/model_{i}.bin' for i in range(5)]
#     config = MeanV2BaseConfig()
#     meanmodel2f = MeanModel_v2(config)
#     pred_mean_v2f = inference(test_dataloader, MeanBF_2, model=meanmodel2f, with_embedding=False)
#     # svr_preds = embedding_svr_test(embeddings, len(test_df))
#     del meanmodel2f
#     reset_memory()
    
    
    #### mean large v2: 
#     MeanL_3 = [ f'/kaggle/input/newmeanlarge/model_{i}.bin' for i in range(5)]
#     config = MeanV2LargeConfig()
#     meanmodel3 = MeanModel_v2(config)
#     pred_mean_v3 = inference(test_dataloader, MeanL_3, model=meanmodel3, with_embedding=False)
#     # svr_preds = embedding_svr_test(embeddings, len(test_df))
#     del meanmodel3
#     reset_memory()
    

    #### attn large 1:  
#     AttL_1 = [ f'/kaggle/input/largeattnlit/model_{i}.bin' for i in range(5)]
#     config = AttConfig1()
#     attmodel1 = AttModel(config)
#     pred_attL_v1 = inference(test_dataloader, AttL_1, model=attmodel1, with_embedding=False)
#     # svr_preds = embedding_svr_test(embeddings, len(test_df))
#     del attmodel1
#     reset_memory()
    
    
    #### attn base 2: 
    AttB_2 = [ f'/kaggle/input/meanattnreinit/model_{i}.bin' for i in range(5)]
    config = AttConfig2()
    attmodel2 = AttModel2(config)
    pred_attL_v2 = inference(test_dataloader, AttL_2, model=attmodel2, with_embedding=False)
    # svr_preds = embedding_svr_test(embeddings, len(test_df))
    del attmodel2
    reset_memory()
    
    # ................................................

    
    ################### Post Process ########################
    predictions = pred_attL_v2
#     predictions = svr_preds * 0.5 + pred_lit * 0.5  # or whatever


    ################### Submisson ########################
    result_df = pd.DataFrame({'id': test_df.id, 'target': predictions})
    result_df.to_csv('submission.csv', index=False)
    print(result_df.head(10))

#### Only SVR inference[Test using]

-   id    target
- 0  c0f722661 -0.367501
- 1  f0953f0a5 -0.398820
- 2  0df072751 -0.537474
- 3  04caf4e0c -2.299636
- 4  0e63f8bea -1.962926
- 5  12537fe78 -1.093182
- 6  965e592c0  0.077126

问题在于 需要合适的 embedding， 容易过拟合，需要bagging

In [ ]:
# # rmse_score SVR
# from joblib import dump, load
# from sklearn.svm import SVR
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import StratifiedKFold


# def rmse_score(y_true,y_pred):
#     return np.sqrt(mean_squared_error(y_true,y_pred))


# def embedding_svr_test(df, save_dir, bert_path, bert_nums=5, svr_nfolds=10):
#     # get embeddings
#     models_embedding = []
#     for fold_num in range(bert_nums):
#         print(f'{by_}{r_}  Model#{fold_num+1} inferencing {sr_}')
#         device = Config.device

#         test_dataloader = get_test_data(df)

#         model = MeanModelEmbedding()
#         model.load_state_dict(torch.load(bert_path + f'model_{fold_num}.bin'))
#         model.to(device)
#         model.eval()

#         embeddings = []
#         with torch.no_grad():
#             for i, batch in tqdm(enumerate(test_dataloader)):
#                 sent_id, mask = batch['input_ids'].to(Config.device), batch['attention_mask'].to(Config.device)
#                 outputs = model(sent_id, mask)
#                 outputs = outputs.detach().cpu().numpy()
#                 embeddings.extend(outputs)
#             embeddings = np.array(embeddings)
#         models_embedding.append(embeddings)

#         del model
#         gc.collect()
#         torch.cuda.empty_cache()
        
#     print(f'Embedding got.')

#     # SVM predict: 5 SVR model
#     results = np.zeros((df.shape[0]))
#     for index, X_test in enumerate(models_embedding):
#         print(f'{by_}{r_}  SVR#{index+1} predicting {sr_}')
#         for i in range(svr_nfolds):
#             svr = load(save_dir + f'svr_{index}_{i}.bin')
#             preds = svr.predict(X_test)
#             results += preds
            
#     print(f'Complete.')

#     return results / bert_nums / svr_nfolds